In [16]:
# setup library imports
import io, time, json
import requests
import pandas as pd
import numpy as np

from pathlib import Path
from bs4 import BeautifulSoup

In [17]:
#Read the Yelp API key from file
def read_api_key(filepath="api_key.txt"):
    return Path(filepath).read_text().strip()

In [18]:
api_key = read_api_key()
api_key

'Y0vpAcCzpLY3l5VSChBzAcRpy-JrWmmaOenfUf-AGrC4lKtc79YDH503ZZSURFVGsAx_I1-Xo0T6YykBPmaOalvnGubVhpIH_K0kfIcWEh0FLftyNyUQ75MXaW0wYHYx'

In [19]:
#return the raw HTML for the specified URL
def retrieve_html(url):
    r = requests.get(url, auth=('user', 'pass'))   
    return (r.status_code, r.text)

In [20]:
#Make an authenticated request to the Yelp API.
def yelp_search(api_key, term, location):
    
    search_url = "https://api.yelp.com/v3/businesses/search"
    
    params = {"location": location, "term": term, "categories" : "restaurants"}
    
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    response = requests.get(search_url, params = params, headers = headers)
    
    data = json.loads(response.text)
    
    return (data["total"], data["businesses"])


In [21]:
(total, businesses) = yelp_search(api_key, "taco", "University District, Seattle")
total

156

In [22]:
#Retrieve ALL the restaurants on Yelp for a given query.
#returns the api response as a list of dictionaries
#max number of responses is 1000
def all_restaurants(api_key, term, location):
    search_url = "https://api.yelp.com/v3/businesses/search"
    
    params1 = {"location": location, "term": term, "categories" : "restaurants"}
    
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    response1 = requests.get(search_url, params = params1, headers = headers)   
    data1 = json.loads(response1.text)
    
    #Note that only 1000 records can be obtained at one time
    records_num = data1['total']    
    requests_num = records_num//20 + 1
    offset = 0
    result = []
    
    for i in range(requests_num):
        #20 restaurants each request
        curr_offset = offset + i*20
        params = {"location": location, "term": term, "offset": curr_offset, "categories" : "restaurants"}
        response = requests.get(search_url, params = params, headers = headers)
        data = json.loads(response.text)
        result += data["businesses"]
    #pause slightly between requests
        time.sleep(.300)
        
    return result

In [23]:
tacos = all_restaurants(api_key, "taco", "University District, Seattle")
type(tacos)

list

In [24]:
print(len(tacos))
tacos[6]

156


{'id': 'ZTFVwQaFVhp_4BU33xoclg',
 'alias': 'the-counter-at-old-ballard-catering-seattle',
 'name': 'The Counter At Old Ballard Catering',
 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/4pqkrTkLd8Fgn1FYieyB9g/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/the-counter-at-old-ballard-catering-seattle?adjust_creative=Yd84IPqpgzteXDQ2QE83uA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Yd84IPqpgzteXDQ2QE83uA',
 'review_count': 36,
 'categories': [{'alias': 'newamerican', 'title': 'American (New)'},
  {'alias': 'greek', 'title': 'Greek'},
  {'alias': 'tacos', 'title': 'Tacos'}],
 'rating': 5.0,
 'coordinates': {'latitude': 47.675679, 'longitude': -122.311566},
 'transactions': ['pickup', 'delivery'],
 'location': {'address1': '1505 NE 65th St',
  'address2': '',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98115',
  'country': 'US',
  'state': 'WA',
  'display_address': ['1505 NE 65th St', 'Seattle, WA 98115']},
 'phone': '+120625747

In [25]:
#parse the api response into a pandas dataframe
#api response is all the restaurants matched
def parse_api_response(api_response):
    
    df = pd.DataFrame(columns=('name','category','latitude','longitude','price','rating','url', 'review_count'))
    
    for i in range(len(api_response)): 
        
        restaurant = api_response[i]
        name = restaurant['name']
        category_list = []
        for category in restaurant['categories']:
            category_list.append(category['alias'])
        seperator = ','
        category_string = seperator.join(category_list)
        
        coordinates = restaurant['coordinates']
        latitude = coordinates['latitude']
        longitude = coordinates['longitude']
        
        if 'price' in restaurant:
            price = restaurant['price']
            price_indicator = price.count('$')
        else:
            price_indicator = 'NA'
        
        rating = restaurant['rating']
        url = restaurant['url']
        review_count = restaurant['review_count']
            
        df.loc[i] = [name, category_string, latitude, longitude, price_indicator, rating, url, review_count]
    
    return df

In [26]:
taco_restaurants_df = parse_api_response(tacos)
taco_restaurants_df

,name,category,latitude,longitude,price,rating,url,review_count
0,El Camion,"foodtrucks,mexican",47.661607,-122.287411,1,4.0,https://www.yelp.com/biz/el-camion-seattle-16?...,82
1,Off the Rez,"foodtrucks,burgers,tacos",47.659920,-122.311825,1,4.0,https://www.yelp.com/biz/off-the-rez-seattle?a...,195
2,Guanaco's Tacos Pupuseria,"salvadoran,tacos",47.657141,-122.314029,2,4.0,https://www.yelp.com/biz/guanacos-tacos-pupuse...,338
3,TNT Taqueria,mexican,47.661509,-122.332940,1,4.0,https://www.yelp.com/biz/tnt-taqueria-seattle?...,444
4,Agua Verde Cafe,"mexican,venues,breakfast_brunch",47.651610,-122.314410,2,3.5,https://www.yelp.com/biz/agua-verde-cafe-seatt...,996
...,...,...,...,...,...,...,...,...
151,The Lodge Sports Grille,"sportsbars,tradamerican",47.690728,-122.355599,2,2.5,https://www.yelp.com/biz/the-lodge-sports-gril...,198
152,Revel,"korean,newamerican,cocktailbars",47.652033,-122.354123,2,3.5,https://www.yelp.com/biz/revel-seattle?adjust_...,1386
153,Portage Bay Cafe - Roosevelt,"newamerican,breakfast_brunch,cafes",47.657570,-122.317600,2,4.0,https://www.yelp.com/biz/portage-bay-cafe-roos...,2427
154,Ivar's Salmon House,"seafood,lounges,tradamerican",47.653620,-122.324040,2,3.5,https://www.yelp.com/biz/ivars-salmon-house-se...,1228


In [27]:
#Parse the reviews on a single page of a restaurant.
def parse_page(html):
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify())
    review_soups = soup.find_all("script",type="application/ld+json")
    
    description_list = []
    for soup in review_soups:
        #bs4 uses .string, not .text
        text = soup.string
        #decode the json into python dict
        js_dict = json.loads(text)
        
        if 'review' in js_dict:
            review_list = js_dict['review']
        
            for i in range(len(review_list)):
                review_dict = review_list[i]
                description_list.append(review_dict['description'])

    return description_list

In [28]:
#retrieve ALL of the reviews for a single restaurant on Yelp.
#returns: reviews (list): list of dictionaries containing extracted review information
def extract_reviews(url):
    
    api_url = url + '%3Fstart%3D40'
    
    html_obj = retrieve_html(url)[1]
    
    review_list = parse_page(html_obj)
    
    result = review_list
    
    return result


In [29]:
test_url = taco_restaurants_df.loc[1]['url']
test_url

'https://www.yelp.com/biz/off-the-rez-seattle?adjust_creative=Yd84IPqpgzteXDQ2QE83uA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=Yd84IPqpgzteXDQ2QE83uA'

In [30]:
test_reviews = extract_reviews(test_url)

print(len(test_reviews))

test_reviews

20


['Exceptional service! It was fun watching him gently knead the dough and then dip it in the fryer. Husband had a bison taco and I had pulled pork and both were outstanding! He had the fry bread with their house made lemon curd and it was divine. 10 stars!',
 'Off the Rez is amazing!! 10/10 would always get the bison taco or something sweet!! The coffee is amazing also!',
 'This is interesting combo: Indian flatbread with contemporary Taco.  I ordered bison and chili choices and both are pretty good.  Indian Flatbread was fluffy, the meat and the topping mixed well, together was a great intriguing snack.  It was a small taco so you will need 2 for lunch/dinner. \nAlso the cafe is next to the museum so You can tell the entrance staff withiutbbiuyjng the ticket and park the lot in front of the museum and cafe.  Pay by mobile or the machine for parking fee (the day we visited, the machine was broken, we paid and extended hours by mobile app).  \nThe staffs were nice and the cafe was clean

In [34]:
url_list = taco_restaurants_df['url'].tolist()

In [36]:
result = []
for i in range(len(url_list)):
    url = url_list[i]
    print(i)
    result += extract_reviews(url)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155


In [38]:
len(result)

2564

In [45]:
result[1]

'I have heard legend of the burritos and other food here, but have had less opportunity to venture out since the onset of COVID. Thankfully, El Camion is available on DoorDash to help make things feel a little more normal. I had the steak quesadilla and trust and believe this was the fattest quesadilla I ever did see and it was incredible. Even a clean plate club like me couldn&apos;t finish this; it reheated pretty well the next day but I&apos;d recommend splitting something.'

In [56]:
import csv
with open('reviews.csv', 'w') as f:
    writer = csv.writer(f) 
    writer.writerow(result)
    f.close()


In [58]:
with open('reviews.csv', 'r') as f:
    reader = csv.reader(f) 
    for row in reader:
        text = ','.join(row)
        
type(text)


str

In [51]:
data = [1,2,3,4,5]
with open('numbers.csv', 'w') as f:
    writer = csv.writer(f) 
    writer.writerow(data)
    f.close()
    


In [54]:
with open('numbers.csv', 'r') as f:
    reader = csv.reader(f) 
    for row in reader:
        str1 = ','.join(row)

str1
        

'1,2,3,4,5'